In [1]:

from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
import os


from sklearn.model_selection import train_test_split
from pathlib import Path
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

from datetime import datetime

from winn_dixie_recpt_parser import WinnDixieRecptParser 

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

print(os.getcwd())


C:\Users\steve\OneDrive - NOLA Business IT\source\repos\grocery-ml


In [2]:
def show_grouped(grouped, rows=10):
    # collect only the daysSinceLastPurchase_* columns
    feature_cols = [c for c in grouped.columns if c.startswith("daysSinceLastPurchase_")]

    for i in range(min(rows, len(grouped))):
        print("Row:", i)
        print("Date:", grouped.iloc[i]["date"])
        print("Time:", grouped.iloc[i]["time"])
        print("Items:", grouped.iloc[i]["item"])
        print("------ daysSinceLastPurchase ------")

        for col in feature_cols:
            print(f"{col}: {grouped.iloc[i][col]}")

        print("-----------------------------------")


def show_encoded(encoded_df, rows=10):
    # Identify columns
    days_cols = [c for c in encoded_df.columns if c.startswith("daysSinceLastPurchase_")]
    item_cols = [c for c in encoded_df.columns if c not in days_cols and c not in ["date", "time"]]

    for i in range(min(rows, len(encoded_df))):
        print("Row:", i)
        print("Date:", encoded_df.iloc[i]["date"])
        print("Time:", encoded_df.iloc[i]["time"])

        # Show the items purchased (reverse one-hot)
        purchased_items = []
        row_vals = encoded_df.iloc[i]

        for item in item_cols:
            if row_vals[item] == 1:
                purchased_items.append(item)

        print("Items:", purchased_items)

        print("------ daysSinceLastPurchase ------")
        for col in days_cols:
            print(f"{col}: {encoded_df.iloc[i][col]}")

        print("-----------------------------------")


In [3]:

rows = []

recptParser  = WinnDixieRecptParser();

for p in Path("StevePhone2/pdf/text").glob("*.txt"):
    result = recptParser.parse(p.read_text(encoding="utf-8", errors="ignore"))
    for r in result["items"]:
        rows.append({
            "source": p.name,
            "date": result["date"],
            "time": result["time"],
            "manager": result["manager"],
            "cashier": result["cashier"],
            "item": r["item"],
            "qty": r["qty"],
            "reg": r["reg"],
            "youPay": r["youPay"],
            "reportedItemsSold": result["reported"],
            #"rowsMatchReported": result["validation"]["rowsMatchReported"],
            "qtyMatchReported": result["validation"]["qtyMatchReported"],
        })

df = pd.DataFrame(rows)


In [4]:
# Clean Item list

df["item"] = (
    df["item"]
        .astype(str)
        .str.strip()      
        .str.replace(r"\s+", "-", regex=True)     # collapse ANY whitespace into ONE hyphen
        .str.replace("[^A-Za-z0-9-]", "", regex=True)  # keep A-Z, a-z, 0-9 and hyphens
        .str.lower()
)


In [5]:
# Transpose / group
df["date"] = pd.to_datetime(df["date"])
df["time"] = df["time"].astype(str)
df = df.sort_values(["date", "time"])
grouped = (df.groupby(["date", "time"])["item"].apply(list).reset_index())

# unique items
unique_items = set()
for item_list in grouped["item"]:
    for item in item_list:
        unique_items.add(item)
unique_items = sorted(unique_items)

# create empty daysSince columns (all zero for now)
for item in unique_items:
    grouped["daysSinceLastPurchase_" + item] = 0

# ------------ NEW CODE YOU NEED HERE ------------
# Fill the daysSinceLastPurchase columns
for item in unique_items:
    last_date_seen = None
    values = []

    for idx in range(len(grouped)):
        current_date = grouped.iloc[idx]["date"]
        items_this_trip = grouped.iloc[idx]["item"]

        # If item purchased today → update last seen
        if item in items_this_trip:
            last_date_seen = current_date

        # Compute days difference
        if last_date_seen is None:
            values.append(0)   # first occurrence → okay to use 0 as you decided
        else:
            days = (current_date - last_date_seen).days
            values.append(days)

    grouped["daysSinceLastPurchase_" + item] = values
# -------------------------------------------------

# build index mapping
item_to_index = {}
for i in range(len(unique_items)):
    item_to_index[unique_items[i]] = i

##print(grouped.columns.tolist())


C:\Users\steve\AppData\Local\Temp\ipykernel_20052\2973701663.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped["daysSinceLastPurchase_" + item] = 0
C:\Users\steve\AppData\Local\Temp\ipykernel_20052\2973701663.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped["daysSinceLastPurchase_" + item] = 0
C:\Users\steve\AppData\Local\Temp\ipykernel_20052\2973701663.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

In [6]:
num_items = len(unique_items)
vectors = []

for item_list in grouped["item"]:
    vector = np.zeros(num_items, dtype=np.int32)

    for name in item_list:
        index = item_to_index[name]
        vector[index] = 1

    vectors.append(vector)

encoded_df = pd.DataFrame(vectors, columns=unique_items)
days_cols = [c for c in grouped.columns if c.startswith("daysSinceLastPurchase_")]

encoded_df = pd.concat(
    [grouped[["date", "time"]], grouped[days_cols], encoded_df],
    axis=1
)

#show_encoded(encoded_df)

In [7]:
# fix time date

encoded_df["dateTime"] = pd.to_datetime(
    encoded_df["date"].astype(str) + " " + encoded_df["time"].astype(str),
    errors="coerce"
)

encoded_df = encoded_df.drop(columns=["date", "time"])

dt = encoded_df["dateTime"]

encoded_df["year"]    = dt.dt.year
encoded_df["month"]   = dt.dt.month
encoded_df["day"]     = dt.dt.day
encoded_df["hour"]    = dt.dt.hour
encoded_df["minute"]  = dt.dt.minute
encoded_df["dow"]     = dt.dt.dayofweek
encoded_df["doy"]     = dt.dt.dayofyear
encoded_df["quarter"] = dt.dt.quarter


encoded_df["hour_sin"] = np.sin(2 * np.pi * encoded_df["hour"] / 24.0)
encoded_df["hour_cos"] = np.cos(2 * np.pi * encoded_df["hour"] / 24.0)

encoded_df["minute_sin"] = np.sin(2 * np.pi * encoded_df["minute"] / 60.0)
encoded_df["minute_cos"] = np.cos(2 * np.pi * encoded_df["minute"] / 60.0)

encoded_df["dow_sin"] = np.sin(2 * np.pi * encoded_df["dow"] / 7.0)
encoded_df["dow_cos"] = np.cos(2 * np.pi * encoded_df["dow"] / 7.0)
encoded_df["month_sin"] = np.sin(2 * np.pi * encoded_df["month"] / 12.0)
encoded_df["month_cos"] = np.cos(2 * np.pi * encoded_df["month"] / 12.0)

encoded_df["doy_sin"] = np.sin(2 * np.pi * encoded_df["doy"] / 365.0)
encoded_df["doy_cos"] = np.cos(2 * np.pi * encoded_df["doy"] / 365.0)

cols_to_drop = ["month", "hour", "minute", "dow", "doy"]
encoded_df = encoded_df.drop(columns=cols_to_drop)

noncyc_cols = ["year", "day", "quarter"]

for col in noncyc_cols:
    mean = encoded_df[col].mean()
    std = encoded_df[col].std()

    if std == 0:
        std = 1.0

    encoded_df[col + "_norm"] = (encoded_df[col] - mean) / std

encoded_df = encoded_df.drop(columns=["year", "day", "quarter"])
encoded_df = encoded_df.drop(columns=["dateTime"])


C:\Users\steve\AppData\Local\Temp\ipykernel_20052\822516727.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  encoded_df["dateTime"] = pd.to_datetime(


# TRAIN !!!

In [8]:
time_feature_cols = [
    "hour_sin", "hour_cos",
    "minute_sin", "minute_cos",
    "dow_sin", "dow_cos",
    "month_sin", "month_cos",
    "doy_sin", "doy_cos",
    "year_norm", "day_norm", "quarter_norm",
]

exclude = set(time_feature_cols + ["date", "time", "dateTime"])  # whatever time/meta cols still exist
item_cols = [c for c in encoded_df.columns if c not in exclude]

X = encoded_df[time_feature_cols].to_numpy(dtype=np.float32)   # shape: (num_trips, num_time_features)
y = encoded_df[item_cols].to_numpy(dtype=np.float32)          # shape: (num_trips, num_items)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,       # 20% test set
    random_state=42,
    shuffle=True
)

import sys
print(sys.executable)

import tensorflow as tf
from tensorflow.keras import layers, models

input_dim = X_train.shape[1]     # number of time features (≈ 13)
output_dim = y_train.shape[1]    # number of items

model = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(output_dim, activation="sigmoid")  
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="bin_acc"),
        tf.keras.metrics.AUC(curve="ROC", name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


C:\Users\steve\.conda\envs\intercard-ml\python.exe
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - auc: 0.5453 - bin_acc: 0.3700 - loss: 0.1975 - precision: 0.3308 - recall: 0.5494 - val_auc: 0.6463 - val_bin_acc: 0.3746 - val_loss: -1.1580 - val_precision: 0.4598 - val_recall: 0.6261
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - auc: 0.6603 - bin_acc: 0.4067 - loss: -0.9935 - precision: 0.4016 - recall: 0.6621 - val_auc: 0.7512 - val_bin_acc: 0.4155 - val_loss: -2.8593 - val_precision: 0.5390 - val_recall: 0.7278
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - auc: 0.7511 - bin_acc: 0.4376 - loss: -2.3324 - precision: 0.4622 - recall: 0.7584 - val_auc: 0.8265 - val_bin_acc: 0.4504 - val_loss: -4.9720 - val_precision: 0.6088 - val_recall: 0.8153
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - auc: 0.8162 - bin_acc: 0.4664 - loss: -4.0833 - precision: 0.5157 - recall: 0.8352 - val_auc: 0.8726 - val_bin_acc: 0.4742 - val_loss: -7.6049 - val_precision: 0.6565 - val_recall: 